# Combine all the Output files

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Path to the folder containing the CSV files
outputfolderLoc = r"/home/dcil/Desktop/Abhi/CrackDetection/ExcelData/Output/Manhatten_Batch1_OUT/CORRECTED"

# List to hold individual DataFrames
all_csvs = []

# Loop through all files in the directory
for file in tqdm(os.listdir(outputfolderLoc)):
    if file.endswith(".csv"):
        csv_path = os.path.join(outputfolderLoc, file)
        try:
            df = pd.read_csv(csv_path)
            all_csvs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Combine all DataFrames into one
combined_df = pd.concat(all_csvs, ignore_index=True)

# Output file path
output_csv_path = os.path.join(outputfolderLoc, "combined_output.csv")

# Save the combined DataFrame
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined CSV saved to: {output_csv_path}")


# Plot detection with severity red high, green low , yellow mod

In [3]:
import os
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Paths
contour_csv_path = r"/home/dcil/Desktop/Abhi/CrackDetection/ExcelData/Output/6files_OUT/CORRECTED/196_8.csv"
metadata_csv_path = r"/home/dcil/Desktop/Abhi/CrackDetection/ExcelData/Input/6files.csv"
output_folder = r"/run/user/1000/gvfs/smb-share:server=10.15.1.24,share=video%202/Software department data/SANJAY/OFFICE WORK/Cracks/Outputs/ICC_05_08_2025"
os.makedirs(output_folder, exist_ok=True)

# Generate dynamic colors for classes
def get_class_colors(n_classes):
    cmap = plt.get_cmap('tab20')  # good for up to 20 classes
    colors = {}
    for i in range(n_classes):
        color = cmap(i / n_classes)  # RGBA normalized
        bgr = (
            int(color[2] * 255),
            int(color[1] * 255),
            int(color[0] * 255)
        )
        colors[i] = bgr
    return colors

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load metadata
df_meta = pd.read_csv(metadata_csv_path)

# Parameters
chunk_size = 100000
missing_images_log = []
chunk_idx = 0

# Process CSV in chunks
for chunk in pd.read_csv(contour_csv_path, chunksize=chunk_size):
    chunk_idx += 1
    print(f"Processing chunk {chunk_idx}...")

    # Merge contour data with metadata
    df_merged = pd.merge(chunk, df_meta, on=["IDSourceFrame", "IDSession"], how="left")

    # Group by image identifiers
    grouped = df_merged.groupby(["IDSourceFrame", "IDSession"])
    for (id_frame, id_session), group in tqdm(grouped, desc=f"Chunk {chunk_idx} - Processing images"):

        image_path = group['paths'].iloc[0]
        image_name = group['ImageName'].iloc[0]

        # Validate image path
        if not isinstance(image_path, str) or not os.path.exists(image_path):
            missing_images_log.append((id_frame, id_session, image_path))
            continue

        # Prepare output filename
        safe_name = os.path.basename(image_name)
        image_parts = image_path.split("/")
        saferequiredname = f"{image_parts[-4]}_{safe_name}.jpg"
        save_path = os.path.join(output_folder, saferequiredname)

        # Load or split existing saved image
        if os.path.exists(save_path):
            saved_image = cv2.imread(save_path)
            if saved_image is None:
                print(f"Could not load saved image: {save_path}")
                continue
            w = saved_image.shape[1] // 2
            imgOriginal = saved_image[:, :w]
            image = saved_image[:, w:].copy()
            imgHeight = image.shape[0]
        else:
            image = cv2.imread(image_path)
            if image is None:
                missing_images_log.append((id_frame, id_session, image_path))
                continue
            imgOriginal = image.copy()
            imgHeight = image.shape[0]

        # --- Color Mapping Logic ---
        fixed_colors = {
            0: (0, 255, 0),    # Green
            1: (0, 255, 255),  # Yellow
            2: (0, 0, 255),    # Red
        }

        unique_classes = group["Class"].unique()
        other_classes = [cls for cls in unique_classes if cls not in fixed_colors]
        dynamic_colors = get_class_colors(len(other_classes))

        # Combine fixed + dynamic colors
        class_to_color = {}
        for cls in unique_classes:
            if cls in fixed_colors:
                class_to_color[cls] = fixed_colors[cls]
            else:
                idx = other_classes.index(cls)
                class_to_color[cls] = dynamic_colors[idx]

        # --- Draw contours ---
        for contour_id, contour_group in group.groupby("ContourID"):
            scaleFactor = 0.003999
            X = contour_group["X"].values / scaleFactor
            Y = imgHeight - (contour_group["Y"].values / scaleFactor)

            class_label = contour_group["Class"].iloc[0]
            color = class_to_color.get(class_label, (255, 255, 255))  # fallback white

            pts = np.stack([X, Y], axis=1).astype(np.int32).reshape((-1, 1, 2))

            if len(pts) >= 2:
                cv2.polylines(image, [pts], isClosed=False, color=color, thickness=2)

        # Concatenate original + annotated images
        hConcatImage = cv2.hconcat([imgOriginal, image])
        cv2.imwrite(save_path, hConcatImage)

# Save missing image log
if missing_images_log:
    log_path = os.path.join(output_folder, "missing_images_log.csv")
    pd.DataFrame(missing_images_log, columns=["IDSourceFrame", "IDSession", "MissingPath"]).to_csv(log_path, index=False)


Processing chunk 1...


Chunk 1 - Processing images: 100%|██████████| 8/8 [00:04<00:00,  1.90it/s]


# only detection

In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm

# Paths
contour_csv_path = r"/home/dcil/Desktop/Abhi/CrackDetection/ExcelData/Output/Manhatten_Batch1_OUT/RAW/combined_output.csv"
metadata_csv_path = r"/home/dcil/Desktop/Abhi/CrackDetection/ExcelData/Input/Manhatten_Batch1.csv"
output_folder = r"/run/user/1000/gvfs/smb-share:server=10.15.1.24,share=video%202/Software department data/SANJAY/OFFICE WORK/Cracks/Outputs/ICC_Manhatten_hconcat_3"
anotherwithseverity = r"/run/user/1000/gvfs/smb-share:server=10.15.1.24,share=video%202/Software department data/SANJAY/OFFICE WORK/Cracks/Outputs/ICC_Manhatten_hconcat_3"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Load metadata
df_meta = pd.read_csv(metadata_csv_path)

# Parameters
chunk_size = 100000
missing_images_log = []
chunk_idx = 0

# Process CSV in chunks
for chunk in pd.read_csv(contour_csv_path, chunksize=chunk_size):
    chunk_idx += 1
    print(f"Processing chunk {chunk_idx}...")

    # Merge with metadata
    df_merged = pd.merge(chunk, df_meta, on=["IDSourceFrame", "IDSession"], how="left")

    # Progress bar over grouped image data
    grouped = df_merged.groupby(["IDSourceFrame", "IDSession"])
    for (id_frame, id_session), group in tqdm(grouped, desc=f"Chunk {chunk_idx} - Processing images"):
        image_path = group['paths'].iloc[0]
        image_name = group['ImageName'].iloc[0]

        # Skip if image path is invalid or doesn't exist
        if not isinstance(image_path, str) or not os.path.exists(image_path):
            missing_images_log.append((id_frame, id_session, image_path))
            continue

        # Get image name and output filename
        safe_name = os.path.basename(image_name)
        image_parts = image_path.split("/")
        saferequiredname = f"{image_parts[-4]}_{safe_name}.jpg"
        save_path = os.path.join(output_folder, saferequiredname)

        # Check if image is already saved (reopen and draw again)
        if os.path.exists(save_path):
            saved_image = cv2.imread(save_path)
            if saved_image is None:
                print(f"Could not load existing saved image: {save_path}")
                continue

            # Split into left and right halves
            w = saved_image.shape[1] // 2
            imgOriginal = saved_image[:, :w]
            image = saved_image[:, w:].copy()
        else:
            # Load fresh image
            image = cv2.imread(image_path)
            if image is None:
                missing_images_log.append((id_frame, id_session, image_path))
                continue
            imgOriginal = image.copy()

        # Draw contours on the right half image
        for contour_id, contour_group in group.groupby("ContourID"):
            pts = contour_group[['PixelX', 'PixelY']].values.astype(np.int32).reshape((-1, 1, 2))
            if len(pts) >= 2:
                cv2.polylines(image, [pts], isClosed=False, color=(0, 0, 255), thickness=2)

        # Concatenate and save
        hConcatImage = cv2.hconcat([imgOriginal, image])
        cv2.imwrite(save_path, hConcatImage)

# Save missing image logs
if missing_images_log:
    log_path = os.path.join(output_folder, "missing_images_log.csv")
    pd.DataFrame(missing_images_log, columns=["IDSourceFrame", "IDSession", "MissingPath"]).to_csv(log_path, index=False)